#### Names of people in the group

Please write the names of the people in your group in the next cell.

Eivind Kjosbakken

In [0]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
!pip install -q ipython_unittest

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
# Loading modules that we need
import unittest
from pyspark.sql.dataframe import DataFrame
from typing import Any

In [0]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame 
def load_df(table_name: "name of the table to load") -> DataFrame:
    return spark.read.parquet(table_name)

users_df = load_df("/user/hive/warehouse/users")
comments_df = load_df("/user/hive/warehouse/comments")
posts_df = load_df("/user/hive/warehouse/posts")


#### Subtask 1: implenenting two helper functions
Impelment these two functions:
1. 'run_query' that gets a Spark SQL query and run it on df which is a Spark DataFrame; it returns the content of the first column of the first row of the DataFrame that is the output of the query;
2. 'run_query2' that is similar to 'run_query' but instead of one DataFrame gets two; it returns the content of the first column of the first row of the DataFrame that is the output of the query.

Note that the result of a Spark SQL query is itself a Spark DataFrame.

In [0]:
def run_query(query: "a SQL query string", df: "the DataFrame that the query will be executed on") -> Any:
    ## YOUR IMPLEMENTATION ##
    df.createOrReplaceTempView("df") #so I can call the df when running a sql query
    ans = spark.sql(query).first()[0] #running the query and only returning first column og first row with .first()[0]
    return ans

def run_query2(query: "a SQL query string", df1: "DataFrame A", df2: "DataFrame B") -> Any:
    ## YOUR IMPLEMENTATION ##
    #same as for run_query over, but with two df's, making sure when using the query I call them "df1" and "df2" (in the queries i wrote further down)
    df1.createOrReplaceTempView("df1")
    df2.createOrReplaceTempView("df2")
    ans = spark.sql(query).first()[0]
    return ans
  

#### Subtask 3: validating the implementations by running the tests

Run the cell below and make sure that all the tests run successfully.

In [0]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

The ipython_unittest extension is already loaded. To reload it, use:
  %reload_ext ipython_unittest


#### Subtask 2: writing a few queries
Write the following queries in SQL to be executed by Spark in the next cell.

1. 'q1': find the 'Id' of the most recently created post ('df' is 'posts_df') 
2. 'q2': find the number users
3. 'q3': find the 'Id' of the user who posted most number of answers
4. 'q4': find the number of questions
5. 'q5': find the display name of the user who posted most number of comments

Note that 'q1' is already available below as an example. Moreover, remmebr that Spark supports ANSI SQL 2003 so your queries have to comply with that standard.

In [0]:
q1 = "SELECT * FROM df ORDER BY CreationDate DESC limit 1"

## YOUR IMPLEMENTATION ##
#from "df" since that is what I called the dataframe in run_query(), count gives me the number of Id's (which are unique, and therefore gives me the number of users)
q2 = "SELECT COUNT(Id) FROM df"

## YOUR IMPLEMENTATION ##
#posttypeid = 2 gives me the answer-posts, ordering descending so the highest number of posts is first, and then just taking the first element with limit 1
q3 = "SELECT OwnerUserId FROM df WHERE PostTypeId=2 GROUP BY OwnerUserId ORDER BY COUNT(Id) DESC limit 1 "

## YOUR IMPLEMENTATION ##
#posttypeid=1 gives me the questions, and then counting each question by their unique identifier Id
q4 = "SELECT COUNT(Id) from df WHERE PostTypeId=1 limit 1"

## YOUR IMPLEMENTATION ##
#joining df1 and df2 by the Id since it is unique, then grouping by displayname so I sum all each unique display names posts, then taking the one with highest number of comments, counting by #PostId #since it is a unique identifier for a comment
q5 = "select DisplayName from df1 inner join df2 on df1.Id=df2.UserId group by DisplayName order by count(PostId) DESC limit 1"

In [0]:
%%unittest_main
class TestTask2(unittest.TestCase):
   
    def test_q1(self):
        # find the id of the most recent post
        r = run_query(q1, posts_df)
        self.assertEqual(r, 95045)

    def test_q2(self):
        # find the number of the users
        r = run_query(q2, users_df)
        self.assertEqual(r, 91616)
    
    def test_q3(self):
        # find the user id of the user who posted most number of answers
        r = run_query(q3, posts_df)
        self.assertEqual(r, 64377)
    
    def test_q4(self):
        # find the number of questions
        r = run_query(q4, posts_df)
        self.assertEqual(r, 28950)

    def test_q5(self):
        # find the display name of the user who posted most number of comments
        r = run_query2(q5, users_df, comments_df)
        self.assertEqual(r, "Neil Slater")

Success.....
----------------------------------------------------------------------
Ran 5 tests in 5.387s

OK
Out[21]: <unittest.runner.TextTestResult run=5 errors=0 failures=0>

#### Subtask 4: answering to questions about Spark related concepts

Please answer the following questions. Write your answer in one to two short paragraphs. Don't copy-paste; instead, write your own understanding.

1. What is the difference between 'DataFrame', 'Dataset', and 'Resilient Distributed Datasets (RDD)'? 
2. When do you suggest using RDDs instead of using DataFrames?
3. What is the main benefit of using DataSets instead of DataFrames?

Write your answers in the next cell.

Your answers:

What is the difference between 'DataFrame', 'Dataset', and 'Resilient Distributed Datasets (RDD)?

First of all, RDD is not organized like dataframe is (dataframe is organized into columns). While RDD is just partitioned elements. Dataset is organized like DataFrame, but has some more functionality like better compile type safety. RDD and Dataset can work on both structured and unstructured data, while DataFrame require the data to be semi-structured or structured. Furthermore you can do different actions and transformations on each of them. Some operations can only be done on RDD, like the map() function. RDD has compile type safety, but lacks some optimization that the dataframe has (but df does not have compile type safety). Datasets have both, but at the same time it has limited with functions, so you might have to convert to dataframe when using them.


When do you suggest using RDDs instead of using DataFrames?

First of all, RDDs work on both structured and unstructured data, while dataframes requires the data to be atleast semistructured (since the data in organized in columns). So if you have data that is not structured, you should use a RDD. RDD should also be used if you want to do certain actions, and transformations on your data, since alot of actions and transformations can only be done on an RDD. For example the map function to manipulate every element in the RDD. Additionally RDD’s are resilient, so in case of losing a partition of the RDD, the complete RDD could be reconstructed. This is what makes the RDD fault-tolerant, so if a node fails in a computation, you can reconstruct the RDD, and then continue on with the computation, without having to restart.


What is the main benefit of using DataSets instead of DataFrames?

Compile-time type safety, the ability to alter data without knowing how the data is structured (the schema of the data). This is the reason DataSet was created in the first place (because of the limitation of compile-time type safety of dataframes, it also gave better optimization). If you for example do something wrong with a variable type, you will be informed when the script is compiled, and not when its ran (so you do not get a runtime error)